Añadiendo imágenes de residuos médicos y residuos eléctronicos del dataset Trashbox a la carpeta de residuos trash para igualar el número de imágenes

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

main_folder="drive/MyDrive/pruebas"

import os
print(os.listdir(main_folder))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
splitfolders.ratio(main_folder, output='dataset-split-unido', seed = 1337, ratio= (0.7,0.2,0.1), group_prefix =  None, move = False)

Copying files: 14899 files [03:35, 69.24 files/s] 


In [ ]:
import os
print(os.listdir('dataset-split-unido'))

['val', 'train', 'test']


In [ ]:
print(os.listdir('dataset-split-unido/train/cardboard'))

['cardboard-trahsnet (34).jpg', 'cardboard 115.jpg', 'cardboard 105.jpg', 'cardboard 596.jpg', 'cardboard 298.jpg', 'cardboard-trahsnet (345).jpg', 'cardboard 589.jpg', 'cardboard 573.jpg', 'cardboard 1745.jpg', 'cardboard 2120.jpg', 'cardboard-trahsnet (263).jpg', 'cardboard 1582.jpg', 'cardboard 1502.jpg', 'cardboard-trahsnet (97).jpg', 'cardboard 1162.jpg', 'cardboard 2222.jpg', 'cardboard 1051.jpg', 'cardboard-trahsnet (308).jpg', 'cardboard 1688.jpg', 'cardboard 530.jpg', 'cardboard 67.jpg', 'cardboard 1531.jpg', 'cardboard-trahsnet (278).jpg', 'cardboard 198.jpg', 'cardboard 1771.jpg', 'cardboard 1184.jpg', 'cardboard 1360.jpg', 'cardboard 2062.jpg', 'cardboard 216.jpg', 'cardboard 323.jpg', 'cardboard 2081.jpg', 'cardboard 1324.jpg', 'cardboard 1346.jpg', 'cardboard-trahsnet (305).jpg', 'cardboard 2106.jpg', 'cardboard-trahsnet (36).jpg', 'cardboard 1455.jpg', 'cardboard 2304.jpg', 'cardboard 1413.jpg', 'cardboard-trahsnet (292).jpg', 'cardboard 2059.jpg', 'cardboard 532.jpg', '

In [ ]:
import cv2
from PIL import Image
import os,sys
import PIL
import numpy as np

classes=['cardboard','paper','plastic','trash','glass','metal']
X_train = []
Y_train = []
train_directory = 'dataset-split-unido/train'

# Loop through your dataset directory
for class_folder in os.listdir(train_directory):
    class_path = os.path.join(train_directory, class_folder)
    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)
        try:
            img = Image.open(image_path).convert('RGB')
            img = img.resize((64, 64))
            img = np.array(img)
            if img.shape == (64, 64, 3):
              X_train.append((img,class_folder))
              Y_train.append(classes.index(class_folder))
            else:
              print(f"Skipping {image_path} due to incompatible dimensions.")
              print(f"Image shape: {img.shape}")
            # If you have labels, append them to Y_train
        except (PIL.UnidentifiedImageError, OSError):
            print(f"Skipping {image_path} due to an error.")

# Convert the image data to a NumPy array
X_train = np.array(X_train, dtype=object)
Y_train = np.array(Y_train)

X_test = []
Y_test = []
test_directory = 'dataset-split-unido/test'

# Loop through your dataset directory
for class_folder in os.listdir(test_directory):
    class_path = os.path.join(test_directory, class_folder)
    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)
        try:
            img = Image.open(image_path).convert('RGB')
            img = img.resize((64, 64))
            img = np.array(img)
            if img.shape == (64, 64, 3):
              X_test.append((img,class_folder))
              Y_test.append(classes.index(class_folder))
            else:
              print(f"Skipping {image_path} due to incompatible dimensions.")
              print(f"Image shape: {img.shape}")
            # If you have labels, append them to Y_train
        except (PIL.UnidentifiedImageError, OSError):
            print(f"Skipping {image_path} due to an error.")

# Convert the image data to a NumPy array
X_test = np.array(X_test, dtype=object)
Y_test = np.array(Y_test)

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Skipping dataset-split-unido/train/cardboard/cardboard 1075.jpg due to an error.


In [ ]:
X_val = []
Y_val = []
val_directory = 'dataset-split-unido/val'

# Loop through your dataset directory
for class_folder in os.listdir(val_directory):
    class_path = os.path.join(val_directory, class_folder)
    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)
        try:
            img = Image.open(image_path).convert('RGB')
            img = img.resize((64, 64))
            img = np.array(img)
            if img.shape == (64, 64, 3):
              X_val.append((img,class_folder))
              Y_val.append(classes.index(class_folder))
            else:
              print(f"Skipping {image_path} due to incompatible dimensions.")
              print(f"Image shape: {img.shape}")
            # If you have labels, append them to Y_train
        except (PIL.UnidentifiedImageError, OSError):
            print(f"Skipping {image_path} due to an error.")

# Convert the image data to a NumPy array
X_val = np.array(X_val, dtype=object)
Y_val = np.array(Y_val)

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.applications import VGG16
from keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import math

lr_max=0.001
lr_min=0.00001
epochs=100

def cosine_annealing_with_restarts(x):
    lr = lr_max/2 * (1 + math.cos(math.pi * (x % (epochs/5)) / (epochs/5)))
    if lr<lr_min:
        lr=lr_min
    return lr

Y_train_categorical = to_categorical(Y_train, num_classes=6)
Y_val_categorical = to_categorical(Y_val, num_classes=6)
Y_test_categorical = to_categorical(Y_test, num_classes=6)

# Convert your list of tuples to separate lists of images and labels
X_train_images, X_train_labels = zip(*X_train)
X_val_images, X_val_labels = zip(*X_val)
X_test_images, X_test_labels = zip(*X_test)

X_train_images = np.array(X_train_images)
X_val_images = np.array(X_val_images)
X_test_images = np.array(X_test_images)

# loading training data
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        vertical_flip = False,
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        fill_mode='nearest')


train_generator = train_datagen.flow(
        x=X_train_images,
        y=Y_train_categorical,
        batch_size=32,
        shuffle = True)
# loading validation data
val_datagen = ImageDataGenerator(rescale=1./255)



val_generator = val_datagen.flow(
        x=X_val_images,
        y=Y_val_categorical,
        batch_size=32,
        shuffle = True)
# loading testing data
test_datagen = ImageDataGenerator(rescale=1./255)



test_generator = test_datagen.flow(
        x=X_test_images,
        y=Y_test_categorical,
        batch_size=32,
        shuffle = True)

reduce_lr = LearningRateScheduler(cosine_annealing_with_restarts)

# initialising sequential model and adding layers to it
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(BatchNormalization())
cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(BatchNormalization())
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(BatchNormalization())
#cnn.add(base_model)
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(1024, activation='relu'))
cnn.add(BatchNormalization())
cnn.add(tf.keras.layers.Dense(1024, activation='relu'))
cnn.add(BatchNormalization())
cnn.add(tf.keras.layers.Dense(6, activation='softmax'))

opt = Adam(learning_rate=0.01)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
epochs=50
batch_size=128
# finally compile and train the cnn
cnn.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
model_1 = cnn.fit(x=train_generator, validation_data=val_generator, epochs=epochs, callbacks=[reduce_lr,checkpoint])
model = tf.keras.models.load_model('best_model.h5')
score = model.evaluate(test_generator, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/50
326/327 [============================>.] - ETA: 0s - loss: 1.7923 - accuracy: 0.3539
Epoch 1: val_accuracy improved from -inf to 0.25384, saving model to best_model.h5
327/327 [==============================] - 26s 57ms/step - loss: 1.7904 - accuracy: 0.3545 - val_loss: 1.8671 - val_accuracy: 0.2538 - lr: 0.0010
Epoch 2/50


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


326/327 [============================>.] - ETA: 0s - loss: 1.5298 - accuracy: 0.4203
Epoch 2: val_accuracy improved from 0.25384 to 0.40581, saving model to best_model.h5
327/327 [==============================] - 17s 53ms/step - loss: 1.5295 - accuracy: 0.4201 - val_loss: 1.5314 - val_accuracy: 0.4058 - lr: 9.7553e-04
Epoch 3/50
327/327 [==============================] - ETA: 0s - loss: 1.4566 - accuracy: 0.4499
Epoch 3: val_accuracy did not improve from 0.40581
327/327 [==============================] - 18s 56ms/step - loss: 1.4566 - accuracy: 0.4499 - val_loss: 1.9157 - val_accuracy: 0.3283 - lr: 9.0451e-04
Epoch 4/50
326/327 [============================>.] - ETA: 0s - loss: 1.3887 - accuracy: 0.4763
Epoch 4: val_accuracy improved from 0.40581 to 0.48798, saving model to best_model.h5
327/327 [==============================] - 17s 53ms/step - loss: 1.3894 - accuracy: 0.4760 - val_loss: 1.3831 - val_accuracy: 0.4880 - lr: 7.9389e-04
Epoch 5/50
326/327 [============================>.